In [ ]:
ㅈimport pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv("./working/대여소별_대여가능_수량_1시간단위_w_자치구.csv")

In [ ]:
df.head()

,일시,대여소번호,대여소명,대여소자치구,시간대,거치대수량
0,2024-06-01,101,101. (구)합정동 주민센터,마포구,0,0
1,2024-06-01,102,102. 망원역 1번출구 앞,마포구,0,24
2,2024-06-01,103,103. 망원역 2번출구 앞,마포구,0,18
3,2024-06-01,104,104. 합정역 1번출구 앞,마포구,0,1
4,2024-06-01,105,105. 합정역 5번출구 앞,마포구,0,2


In [ ]:
# zero_bike_station_ids는 모든 거치대수가 0인 대여소 - 2024년 06월에 없어진 대여소들을 삭제
zero_bike_station_ids = df.groupby("대여소번호")["거치대수량"].max()
zero_bike_station_ids = zero_bike_station_ids[zero_bike_station_ids == 0].index.tolist()

In [ ]:
# 없어진 대여소 361
len(zero_bike_station_ids)

361

In [ ]:
df1 = df[~df["대여소번호"].isin(zero_bike_station_ids)]

In [ ]:
print(any(df1["대여소번호"].isin(zero_bike_station_ids)))

False


In [ ]:
# 너무 많은 결측치가 존재하는 날짜
# 연속시간대로 결측치가 너무 많이 존재하기 때문에 해당 날짜는 삭제
delete = ['2024-06-16', '2024-06-25', '2024-06-28', '2024-06-29']

In [ ]:
df1 = df1[~df1['일시'].isin(delete)]

In [ ]:
df1.shape

(1639966, 6)

In [ ]:
df_selected = df1[['대여소자치구', '대여소번호', '일시', '시간대', '거치대수량']]

# 누락된 시간들 이전 시간대의 자전거의 수로 결측값 체체
def fill_missing_specific_hour(group, missing_hour, reference_hour):
    group = group.copy()
    hours_present = group['시간대'].tolist()
    if missing_hour not in hours_present and reference_hour in hours_present:
        ref_row = group[group['시간대'] == reference_hour]
        new_row = ref_row.copy()
        new_row['시간대'] = missing_hour
        group = pd.concat([group, new_row], ignore_index=True)
    return group

# 1단계: 8시 누락 시 7시 값으로 체체
df_filled = (
    df_selected
    .groupby(['대여소자치구', '대여소번호', '일시'], group_keys=False)
    .apply(lambda x: fill_missing_specific_hour(x, 8, 7))
)

# 2단계: 11시 누락 시 10시 값으로 체체
df_filled = (
    df_filled
    .groupby(['대여소자치구', '대여소번호', '일시'], group_keys=False)
    .apply(lambda x: fill_missing_specific_hour(x, 11, 10))
)

# 3단계: 5시 누락 시 4시 값으로 체체
df_filled = (
    df_filled
    .groupby(['대여소자치구', '대여소번호', '일시'], group_keys=False)
    .apply(lambda x: fill_missing_specific_hour(x, 5, 4))
)

# 정렬
df_filled = df_filled.sort_values(by=['대여소자치구', '대여소번호', '일시', '시간대']).reset_index(drop=True)


C:\Users\dbtjd\AppData\Local\Temp\ipykernel_10640\330413433.py:19: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: fill_missing_specific_hour(x, 8, 7))
C:\Users\dbtjd\AppData\Local\Temp\ipykernel_10640\330413433.py:26: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: fill_missing_specific_hour(x, 11, 10))
C:\Users\dbtjd\AppData\Local\Temp\ipykernel_10640\330413433.py:33: Depreca

In [ ]:
# 위의 대체 작업중 중복으로 들어간 내용 존재
# 중복제거
df_filled = df_filled.drop_duplicates(subset=['대여소번호', '일시', '시간대'], keep='first')

In [ ]:
df_filled.shape

(1705310, 5)

In [ ]:
# 결측치가 각 날짜마다 랜덤으로 존재하는 대여소
exclude_stations = [4426, 4931, 6059]

# 삭제
df_filtered = df_filled[~df_filled['대여소번호'].isin(exclude_stations)]

In [ ]:
df_filtered.shape

(1704144, 5)

In [ ]:
df_filtered["대여소번호"].nunique()

2731

In [ ]:
df_filtered["일시"].nunique()

26

In [ ]:
df_filtered["시간대"].nunique()

24

In [ ]:
2731 * 26 * 24

1704144

In [ ]:
df_filtered.to_csv("대여소별_대여가능_수량_1시간단위_w_자치구_최종본.csv")